# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [3]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

Then, we define the model, object function and optimizer that we use to classify.

In [4]:
class SimpleNet(nn.Module):
    # TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 8, 5, 2, 2),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, 5, 2, 2),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, 5, 2, 2),
            nn.ReLU(),
            nn.BatchNorm2d(32),
        )
        self.linear = nn.Linear(32*4*4, 10)
        
    def forward(self, input):
        input = self.conv(input)
        input = input.view(input.size(0), -1)
        input = self.linear(input)
        return input
    

    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [5]:
# train and evaluate
step_all = NUM_EPOCHS*len(train_loader)
for epoch in range(NUM_EPOCHS):
    i = 0
    correct = 0
    total_num = 0
    model.train()
    for images, labels in train_loader:
        # TODO:forward + backward + optimize
        i += 1
        step = epoch*len(train_loader) + i 
        output = model(images)
        loss = criterion(output, labels.squeeze())
        loss.backward()
        optimizer.step()
        #if(step%100==0):
        #   print("step[{}/{}], loss: {:.6f}".format(step, step_all, loss.item()))

        output_label = output.data.max(1, keepdim=True)[1]
        correct += output_label.eq(labels.view_as(output_label)).cpu().sum()
        total_num += images.size(0)
            
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    train_accuracy = 100.*float(correct)/float(total_num)
    correct_t = 0
    total_num_t = 0
    test_loss = 0.
    model.eval()
    for images, labels in test_loader:
        # TODO:forward + backward + optimize
        output = model(images)
        output_label = output.data.max(1, keepdim=True)[1]
        correct_t += output_label.eq(labels.view_as(output_label)).cpu().sum()
        total_num_t += images.size(0)
        test_loss += F.cross_entropy(output, labels.squeeze(), size_average=False).item() 
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.*float(correct_t)/float(total_num_t)
    print("epoch[{}/{}], test_loss: {:.6f}, [{}/{}], train_accuracy: {:.6f}%, [{}/{}], test_accuracy: {:.6f}%".format(epoch+1, NUM_EPOCHS, test_loss, correct, total_num, train_accuracy, correct_t, total_num_t, test_accuracy))
    
    


C:\Users\win8\Anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


epoch[1/10], test_loss: 0.368771, [52497/60000], train_accuracy: 87.495000%, [9050/10000], test_accuracy: 90.500000%
epoch[2/10], test_loss: 0.520440, [55347/60000], train_accuracy: 92.245000%, [9420/10000], test_accuracy: 94.200000%
epoch[3/10], test_loss: 0.371319, [56629/60000], train_accuracy: 94.381667%, [9536/10000], test_accuracy: 95.360000%
epoch[4/10], test_loss: 0.613566, [57241/60000], train_accuracy: 95.401667%, [9677/10000], test_accuracy: 96.770000%
epoch[5/10], test_loss: 0.786132, [57812/60000], train_accuracy: 96.353333%, [9556/10000], test_accuracy: 95.560000%
epoch[6/10], test_loss: 0.673561, [57658/60000], train_accuracy: 96.096667%, [9670/10000], test_accuracy: 96.700000%
epoch[7/10], test_loss: 0.670706, [58266/60000], train_accuracy: 97.110000%, [9742/10000], test_accuracy: 97.420000%
epoch[8/10], test_loss: 0.802706, [58187/60000], train_accuracy: 96.978333%, [9723/10000], test_accuracy: 97.230000%
epoch[9/10], test_loss: 0.966765, [58335/60000], train_accuracy:

#### Q5:
Please print the training and testing accuracy.